# Aula 7 - Modelos de Aprendizado

## Aplicar Redes Bayesianas no Dataset Police

Será feito um classificador bayesiano para prever se uma parada resultará ou não em prisão.

In [26]:
import pandas as pd

In [27]:
csv_path = "../data/police.csv"
df = pd.read_csv(csv_path)

In [28]:
from etl.main import PoliceDatasetEtl

police_dataset_etl = PoliceDatasetEtl(df.copy())
featured_df = police_dataset_etl.clean_transform()
search_type_df = police_dataset_etl.get_search_type_df()

### Modelo Probabilístico - Rede Bayesiana

* Objetivo: Prever se uma parada policial resultará em prisão ou não
* Utilizar classificador `Bernoli Naive Bayes`, como a distribuição do `target`(ser preso ou não) segue uma distribuição binária (`Bernouli`)
* Utilizar a proporção 70% - 30% (70% treino, 30% teste)
* Utilizar validação cruzada `k-fold`

#### Notas:
* **Seleção de features:** Dado a possibilidade de reutilização de trabalhos prévios sob esse dataset, as features utilizadas serão as mesmas descobertas como sendo as que possuem melhor relação de `mutual information score` com o *target* `is_arrested`, conforme utilizado no [notebook da aula 4](./etl_aula_4.ipynb). Portanto:
    * search_type
    * search_conducted
    * violation_level
    * driver_race
* **ETL e Feature Engineering:**
    * Está sendo utilizada a class `PoliceDatasetEtl`, desenvolvida durante as atividades anteriores. Essa classe contém métodos de limpeza, extração e engenharia da feature `violation_level` que quantifica a feature original `violation` em uma variável contínua.
* **Comparação:** Como o modelo de árvore de decisão já foi utilizado no [notebook da aula 4](./etl_aula_4.ipynb), faremos uma comparação do desempenho utilizando o modelo bayesiano para a distribuição de Bernouli.

Selecionando e adequando as features

Algumas features, incluindo o `target` deverão ser discretizados uma vez que são representadas por meio de variáveis categóricas. Para isso utilizaremos a técnica de `one hot encoder`.

In [53]:
# Please, note search type is already transformed with "one-hot-encoding" in search_type_df dataset
selected_features = [
    "search_conducted",
    "violation_level",
    "driver_race",
]
target = "is_arrested"

training_df = featured_df[selected_features + [target]].copy()
y = training_df.pop(target)



Aplicando `one-hot-encoder`:

In [54]:

# Fazendo o encoding das features categóricas
encoded_training_df = training_df.copy()
category_feature_to_encode = [
    "driver_race", 
    "search_conducted",
]

# Fazendo o encoding da target
y_encoded = y.copy()
y_encoded = y_encoded.astype("category")
y_encoded = y_encoded.cat.codes

for category in category_feature_to_encode:
    encoded_training_df[category] = encoded_training_df[category].astype("category")
    encoded_training_df[f"{category}_encoded"] = encoded_training_df[category].cat.codes
    
encoded_training_df.pop("driver_race")
encoded_training_df.pop("search_conducted")

# merge training_df with search_type_df
encoded_training_df = encoded_training_df.merge(search_type_df, left_index=True, right_index=True)


Fazendo o split do dataset para treino - teste

In [55]:
from sklearn.model_selection import train_test_split

# split training dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(encoded_training_df, y_encoded, test_size=0.3, random_state=42)

Treinando e medindo a acurácia da `Rede Bayesiana`

In [57]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

# Inicializando o classificador Naive Bayes
naive_bayes_classifier = BernoulliNB()

# Treinando o classificador Naive Bayes com os dados de treino
naive_bayes_classifier.fit(X_train, y_train)

# Fazendo predições com o classificador Naive Bayes
y_pred_test = naive_bayes_classifier.predict(X_test)

# Medindo a acurácia do classificador Naive Bayes com os dados de teste
acuracia = accuracy_score(y_test, y_pred_test)

print(f"Acurácia: {(acuracia * 100):.2f} %")

Acurácia: 95.62 %


Utilizando validação cruzada: `K-fold`